In [1]:
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
day = datetime.datetime.strftime(datetime.datetime.now(), "%Y_%m_%d")
day

'2022_08_26'

In [3]:

! rm -rf "eqList/eqList"$day".xls"
! cp "/Users/ivan/Downloads/eqList"$day".xls" "eqList/eqList"$day".xls"
! ls -l "eqList/"

aeta = f"eqList/eqList{day}.xls"
aeta

total 52216
-rw-r--r--@ 1 ivan  staff   565464  7 30 18:56 eqList2022_07_30.xlsx
-rw-r--r--@ 1 ivan  staff   566439  8  9 01:05 eqList2022_08_09.xlsx
-rw-r--r--@ 1 ivan  staff  3567272  8 11 11:27 eqList2022_08_11.xls
-rw-r--r--@ 1 ivan  staff   566428  8 11 00:45 eqList2022_08_11.xlsx
-rw-r--r--@ 1 ivan  staff  3568288  8 12 16:56 eqList2022_08_12.xls
-rw-r--r--@ 1 ivan  staff  3569648  8 13 15:09 eqList2022_08_13.xls
-rw-r--r--@ 1 ivan  staff  3573668  8 15 16:06 eqList2022_08_15.xls
-rw-r--r--@ 1 ivan  staff  3575011  8 16 18:58 eqList2022_08_16.xls
-rw-r--r--@ 1 ivan  staff  3576341  8 19 17:36 eqList2022_08_19.xls
-rw-r--r--@ 1 ivan  staff  3581086  8 26 10:12 eqList2022_08_26.xls


'eqList/eqList2022_08_26.xls'

In [4]:
N = 3
T = 10
# 经度
lonL, lonH = 98, 107
# 纬度
latL, latH = 22, 34 


In [5]:

import xml
DOMTree = xml.dom.minidom.parse(aeta).documentElement
Row = DOMTree.getElementsByTagName("Row")

cols, data = [], []
for nRow, iRow in enumerate(Row):
    if nRow == 0:
        for jRow in iRow.getElementsByTagName("Cell"):
            cols.append(jRow.childNodes[0].childNodes[0].data)
    else:
        idata = []
        for jRow in iRow.getElementsByTagName("Cell"):
            idata.append(jRow.childNodes[0].childNodes[0].data)
        data.append([float(dj) if nj in [1,2,3,4] else str(dj) for nj ,dj in enumerate(idata)])

data = pd.DataFrame(data, columns=cols)
print(data.dtypes)
data.head(T)


发震时刻       object
震级(M)     float64
纬度(°)     float64
经度(°)     float64
深度(千米)    float64
参考位置       object
dtype: object


,发震时刻,震级(M),纬度(°),经度(°),深度(千米),参考位置
0,2022-08-25 10:52:54,2.4,36.36,114.33,13.0,河北邯郸市磁县
1,2022-08-25 08:00:10,3.9,29.07,105.45,10.0,四川泸州市龙马潭区
2,2022-08-25 01:34:31,3.0,38.47,105.14,15.0,内蒙古阿拉善盟阿拉善左旗
3,2022-08-24 12:12:09,3.1,37.49,96.76,10.0,青海海西州德令哈市
4,2022-08-23 22:31:39,6.3,-5.25,102.85,50.0,印尼苏门答腊岛南部海域
5,2022-08-23 19:12:33,3.3,41.80,81.30,17.0,新疆阿克苏地区拜城县
6,2022-08-23 18:03:36,3.0,41.83,81.28,19.0,新疆阿克苏地区拜城县
7,2022-08-23 08:15:28,3.0,40.16,83.12,17.0,新疆阿克苏地区沙雅县
8,2022-08-22 16:36:33,5.6,-9.25,115.65,130.0,印尼巴厘岛以南海域
9,2022-08-22 12:50:54,3.0,33.20,92.77,10.0,青海玉树州杂多县


In [6]:
def regeo(location):
    try:
        import json
        import requests

        host = 'https://regeo.market.alicloudapi.com'
        path = '/v3/geocode/regeo'
        appcode = '830fa522a512421eaa202bb80afe8921'
        querys = f'location={location}' # 经度在前，纬度在后
        with requests.get(
            f"{host}{path}?{querys}", 
            headers={'Authorization': f'APPCODE {appcode}'}
        ) as r:
            j = json.loads(r.content.decode())["regeocode"]["formatted_address"]
            return j
    except:
        return "NaN"

In [7]:

Kdata = []
for STATION in tqdm(data.head(T)["参考位置"]):
    # idata
    idata = data[data["参考位置"] == STATION][["参考位置", "发震时刻"]]
    idata.rename({
        "参考位置": "初次参考位置", 
        "发震时刻": "初次发震时刻",
    }, axis=1, inplace=True)
    idata["K"] = 1
    # print(idata.shape)

    # jdata
    jdata = data[
        (data["参考位置"] != STATION) & 
        (data["经度(°)"] >= lonL) & (data["经度(°)"] <= lonH) &
        (data["纬度(°)"] >= latL) & (data["纬度(°)"] <= latH) &
        # (data["震级(M)"] >= 3.5) &
        True
    ][["参考位置", "震级(M)", "纬度(°)", "经度(°)", "发震时刻"]]
    jdata["K"] = 1
    # print(jdata.shape)

    # k0data
    k0data = pd.merge(
        idata,
        jdata,
        on="K"
    )
    # print(k0data.shape)

    # k1data
    # 筛选近N天内
    k0data["gap-发震时刻"] = (
        pd.to_datetime(k0data["发震时刻"]) - 
        pd.to_datetime(k0data["初次发震时刻"])
    ).dt.days

    k1data = (k0data["gap-发震时刻"] >= 1) & (k0data["gap-发震时刻"] <= N)
    # print(pd.value_counts(k1data))

    k1data = k0data[k1data]
    # print(k1data.shape)

    # k2data
    k2data = k1data.groupby(
        ["初次参考位置", "初次发震时刻", "K"], 
        as_index=False
    ).agg({
        "震级(M)": "mean", # 特殊/余震
        "纬度(°)": "mean",
        "经度(°)": "mean",
    }).groupby(
        ["K"], 
        as_index=False
    ).agg(
        {
        "K": "sum",
        "震级(M)": "mean", 
        "纬度(°)": "mean",
        "经度(°)": "mean",
    })

    k2data["参考位置"] = STATION
    k2data["K%"] = k2data["K"]/idata.shape[0]
    k2data["震级C"] = k2data["震级(M)"].apply(lambda x: "" if x >= 3.5 else "❌")
    k2data["经度C"] = k2data["经度(°)"].apply(lambda x: "" if x >= lonL and x <= lonH else "❌")
    k2data["纬度C"] = k2data["纬度(°)"].apply(lambda x: "" if x >= latL and x <= latH else "❌")
    k2data["经纬度解析"] = [regeo(f"{_1},{_2}") for _1, _2 in zip(k2data["经度(°)"], k2data["纬度(°)"])]
    k2data["经度(°)"] = k2data["经度(°)"].apply(lambda x: f"{x:.6f}")
    k2data["纬度(°)"] = k2data["纬度(°)"].apply(lambda x: f"{x:.6f}")

    k2data = k2data[
        ['参考位置', '震级(M)', '震级C', '纬度(°)', '纬度C', '经度(°)', '经度C', 'K', 'K%','经纬度解析']
    ]
    # print(k2data)
    Kdata.append(k2data)

    # 
    del idata, jdata, k0data, k1data, k2data


100%|██████████| 10/10 [00:02<00:00,  3.51it/s]


In [8]:
endata = pd.merge(
    data.head(T).rename(
        {icol: f"#{icol}#" for icol in data.columns if icol != "参考位置"}, 
        axis=1
    ), 
    pd.concat(Kdata),
    on="参考位置"
).sort_values("#发震时刻#", ascending=False)

endata

,#发震时刻#,#震级(M)#,#纬度(°)#,#经度(°)#,#深度(千米)#,参考位置,震级(M),震级C,纬度(°),纬度C,经度(°),经度C,K,K%,经纬度解析
0,2022-08-25 10:52:54,2.4,36.36,114.33,13.0,河北邯郸市磁县,3.184444,❌,28.249667,,102.326667,,3,0.750000,四川省凉山彝族自治州喜德县光明镇瓦乃合铺
1,2022-08-25 08:00:10,3.9,29.07,105.45,10.0,四川泸州市龙马潭区,3.070000,❌,28.188000,,101.309000,,5,0.833333,四川省凉山彝族自治州木里藏族自治县卡拉乡烟冈
2,2022-08-25 01:34:31,3.0,38.47,105.14,15.0,内蒙古阿拉善盟阿拉善左旗,3.422121,❌,28.246000,,102.881636,,11,0.578947,四川省凉山彝族自治州昭觉县则普乡来祖库
3,2022-08-24 12:12:09,3.1,37.49,96.76,10.0,青海海西州德令哈市,3.935000,,29.193000,,103.490000,,15,0.789474,四川省乐山市峨边彝族自治县五渡镇猪沟
4,2022-08-23 22:31:39,6.3,-5.25,102.85,50.0,印尼苏门答腊岛南部海域,3.447059,❌,28.931863,,103.330637,,17,0.531250,四川省乐山市马边彝族自治县三河口镇金家沟村
5,2022-08-23 19:12:33,3.3,41.80,81.30,17.0,新疆阿克苏地区拜城县,3.341795,❌,28.255274,,103.448590,,39,0.590909,四川省凉山彝族自治州雷波县马颈子镇大坪
6,2022-08-23 19:12:33,3.3,41.80,81.30,17.0,新疆阿克苏地区拜城县,3.341795,❌,28.255274,,103.448590,,39,0.590909,四川省凉山彝族自治州雷波县马颈子镇大坪
7,2022-08-23 18:03:36,3.0,41.83,81.28,19.0,新疆阿克苏地区拜城县,3.341795,❌,28.255274,,103.448590,,39,0.590909,四川省凉山彝族自治州雷波县马颈子镇大坪
8,2022-08-23 18:03:36,3.0,41.83,81.28,19.0,新疆阿克苏地区拜城县,3.341795,❌,28.255274,,103.448590,,39,0.590909,四川省凉山彝族自治州雷波县马颈子镇大坪
9,2022-08-23 08:15:28,3.0,40.16,83.12,17.0,新疆阿克苏地区沙雅县,3.350333,❌,28.237283,,102.262113,,40,0.645161,四川省凉山彝族自治州冕宁县泸沽镇哇例号铺


In [9]:
# 
day1 = datetime.datetime.strftime(datetime.datetime.now() + datetime.timedelta(days=1), "%Y-%m-%d")
dayn = datetime.datetime.strftime(datetime.datetime.now() + datetime.timedelta(days=N), "%Y-%m-%d")
day7 = datetime.datetime.strftime(datetime.datetime.now() + datetime.timedelta(days=7), "%Y-%m-%d")
print(f">>> {day1} To {day7} / {dayn}")

message = f"""
# 有震预测 earthquake prediction
check_my_prediction(myToken, '{day1}', '{dayn}', 1, latitude=$_1, longitude=$_2, magnitude=$_3)
"""
endata = endata[endata["震级C"] != "❌"]
for _1, _2, _3 in zip(endata["纬度(°)"], endata["经度(°)"], endata["震级(M)"]):
    print(message
          .replace("$_1",f"{float(_1):.6f}")
          .replace("$_2",f"{float(_2):.6f}")
          .replace("$_3",f"{float(_3):.1f}")
         )

message = f"""
# 无震预测 No earthquake prediction
check_my_prediction(myToken, '{day1}', '{day7}', 0)
"""
print(message)
print()


>>> 2022-08-27 To 2022-09-02 / 2022-08-29

# 有震预测 earthquake prediction
check_my_prediction(myToken, '2022-08-27', '2022-08-29', 1, latitude=29.193000, longitude=103.490000, magnitude=3.9)


# 有震预测 earthquake prediction
check_my_prediction(myToken, '2022-08-27', '2022-08-29', 1, latitude=29.070000, longitude=105.450000, magnitude=3.9)


# 无震预测 No earthquake prediction
check_my_prediction(myToken, '2022-08-27', '2022-09-02', 0)




In [10]:
! cp Test002.ipynb "back/Test002-"$day".ipynb"
! ls -l back/

total 216
-rw-r--r--@ 1 ivan  staff  34490  8 16 18:58 Test002-2022_08_16.ipynb
-rw-r--r--@ 1 ivan  staff  33275  8 19 17:36 Test002-2022_08_19.ipynb
-rw-r--r--@ 1 ivan  staff  33096  8 26 10:12 Test002-2022_08_26.ipynb
